#  将 HIN douban movie 数据集 转换为 RSA dbm 数据集

```
下载准备好 HIN 数据集后，运行此 notebook ，将生成 dbm 数据集的 3 个 数据文件。 

utility.cuc_stat.py - INFO - === 数据集评分统计信息 ../data/dbm_ratings.txt ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 1384 *     2070       43873  1.531408  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 ../data/dbm_trust.txt ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 1384 *     1764        2618  0.107235  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 ../data/dbm_meta.txt ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 2070 *       25        2440  4.714976  

修改 configx.py self.dataset_name 以及 cross_validate.py 即可进行实验
```

In [1]:
import sys,os
sys.path.append("..")
from utility.cross_validation import split_5_folds
from utility.cuc_stat import print_dataframe_stats, print_data_file_stats

# 设置好 HIN 路径
HIN_path = "../../HIN/"
# orignal_path = HIN_path + "/Douban Book/" 
orignal_path = HIN_path + "/Douban Movie/" 

def download_rating_from_HIN():
    """
    先下载准备好 HIN 数据集
    """
    if not os.path.isdir(HIN_path):
        print("ERROR: 找不到 HIN 目录； 请先下载 please download HIN dataset first!")
        print("可使用如下命令: ")
        print("cd ../..")
        print("git clone https://github.com/librahu/HIN-Datasets-for-Recommendation-and-Network-Embedding HIN")
    else:
        print ("OK. HIN 目录已发现 ...")

download_rating_from_HIN()

import pandas as pd
from configx.configx import ConfigX

config = ConfigX(dataset_name="db")

print ("\n config 预设值的文件信息为 ")
print (config.trust_path)
print (config.rating_path)
print (config.meta_path)
# print (config.igraph_path)
# print (config.ugraph_path)

print ("\n HIN 路径文件 ", HIN_path)
%ls "$orignal_path"

OK. HIN 目录已发现 ...

 config 预设值的文件信息为 
../data/db_trust.txt
../data/db_ratings.txt
../data/db_meta.txt

 HIN 路径文件  ../../HIN/
movie_actor.dat     movie_type.dat  user_group.dat  user_user.dat
movie_director.dat  README          user_movie.dat


In [2]:
all_rating = {}
all_trust = {}
all_meta = {}

        
def prepare_rating_from_HIN():
    """
    预处理生成 rating 数据
    """

    dpath = orignal_path + "user_movie.dat"
    df = pd.read_csv(dpath, sep='\t', header=None)
    # print ("\n###  %s  统计  ..." % dpath)
    # print_data_num(dpath, df)
    # 将播放数量 进行 max - min 处理
    # df[2] = df[2] / (df[2].max() - df[2].min())
    print ("\n###  %s rating 样例 ..." % orignal_path)
    print(df.head())
#   print ("\n###  写入 %s dataset  ..." % config.rating_path)
#   不覆盖原有的 rating
#     df.to_csv(config.rating_path, sep=' ', header=None, index=None)
    
    print_dataframe_stats(df, dpath=config.rating_path)
    return df

def prepare_trust_from_HIN():
    """
    预处理生成 trust 数据
    """
    dpath = orignal_path + "user_user.dat"
    df = pd.read_csv(dpath, sep='\t', header=None)
    
    # 将播放数量 进行 max - min 处理
    # df[2] = df[2] / (df[2].max() - df[2].min())
    # df[2] = 1 # 补充 社交数据 # douban movie 有数据 1， douban book 没有, 需要补 1； 
    print ("\n###  %s trust 样例 ..." % orignal_path)
    print(df.head())
#     print ("\n###  写入 %s dataset  ..." % config.trust_path)
#   不覆盖原有的 trust
#     df.to_csv(config.trust_path, sep=' ', header=None, index=None)
    print_dataframe_stats(df, dpath=config.trust_path)
    return df

def print_data_num(dpath, data):
    """
    旧的打印函数，不再使用了
    """
    print("total(rating/...): ", len(data))
    print("col0 (user) \tnum: \t", len(data[0].unique()))
    print("col1 (item/user/...) num: ", len(data[1].unique()))
    

all_rating = prepare_rating_from_HIN()
all_trust = prepare_trust_from_HIN()

utility.cuc_stat.py - INFO - === 数据集评分统计信息 ../data/db_ratings.txt ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 13367 *    12677     1068278  0.630426  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 ../data/db_trust.txt ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 2440 *     2294        4085  0.072981  



###  ../../HIN//Douban Movie/ rating 样例 ...
      0  1  2
0  2681  1  2
1  3574  1  4
2  7078  1  4
3  9782  1  3
4  9885  1  3

###  ../../HIN//Douban Movie/ trust 样例 ...
       0   1  2
0     25   2  1
1    448   9  1
2  12574  10  1
3  11469  15  1
4   5728  17  1


In [3]:
%ls "$orignal_path/"

movie_actor.dat     movie_type.dat  user_group.dat  user_user.dat
movie_director.dat  README          user_movie.dat


## 默认使用的是 movie_type，  还可使用的 movie_actor, movie_director 数据，对比推荐效果

In [4]:
def prepare_meta_from_HIN(dpath):
    """
    预处理生成 meta 数据
    """
    df = pd.read_csv(dpath, sep='\t', header=None)
    print ("\n###  %s  统计  ..." % dpath)
#     print_data_num(dpath, df)
    # 将播放数量 进行 max - min 处理
    # df[2] = df[2] / (df[2].max() - df[2].min())
#     df[2] = 1 # 补充 社交数据 # douban movie 有数据 1， douban book 没有, 需要补 1； 
    print ("\n###  %s dataset 样例 ..." % orignal_path)
    print(df.head())
    print ("\n###  !!! 从 %s 覆盖 写入 %s dataset  ..." % (dpath, config.meta_path))
    df.to_csv(config.meta_path, sep=' ', header=None, index=None)
    print_dataframe_stats(df, dpath = dpath)
    return df 

# 打印并写入不同文件， 最后写入的为 movie_type
# prepare_meta_from_HIN(orignal_path + "movie_actor.dat")
# prepare_meta_from_HIN(orignal_path + "movie_director.dat")
all_meta = prepare_meta_from_HIN(orignal_path + "movie_type.dat")


###  ../../HIN//Douban Movie/movie_type.dat  统计  ...

###  ../../HIN//Douban Movie/ dataset 样例 ...
       0  1  2
0   2087  1  1
1   7001  1  1
2  10099  1  1
3  10100  1  1
4  10125  1  1

###  !!! 从 ../../HIN//Douban Movie/movie_type.dat 覆盖 写入 ../data/db_meta.txt dataset  ...


utility.cuc_stat.py - INFO - === 数据集评分统计信息 ../../HIN//Douban Movie/movie_type.dat ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 12676 *       38       27668  5.743967  


In [5]:
print_dataframe_stats(all_rating, dpath="rating")
print_dataframe_stats(all_trust, dpath="trust")
print_dataframe_stats(all_meta, dpath="meta")

utility.cuc_stat.py - INFO - === 数据集评分统计信息 rating ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 13367 *    12677     1068278  0.630426  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 trust ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 2440 *     2294        4085  0.072981  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 meta ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 12676 *       38       27668  5.743967  


## 参考 filetrust (m *  n = 1,508	* 2,071) 数据大小进行裁剪




In [6]:

len(all_trust[1].unique())
dbm_trust= all_trust[all_trust[0].isin(all_trust[1])]
print("交集后的有信任用户数: " , dbm_trust[0].unique().size)


dbm_rating_trust = all_rating[all_rating[0].isin(dbm_trust[0])]
print_dataframe_stats(dbm_rating_trust, dpath="dbm 保留有 trust 数据的rating")
print_dataframe_stats(dbm_trust, dpath="dbm 保留都有 trust 数据的 trust")

交集后的有信任用户数:  1592


utility.cuc_stat.py - INFO - === 数据集评分统计信息 dbm 保留有 trust 数据的rating ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 1592 *     9941      242576  1.532762  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 dbm 保留都有 trust 数据的 trust ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 1592 *     1936        2960  0.096038  


In [7]:
print ("对 meta serail 进行排序与汇总")
meta_serial = all_meta.groupby(all_meta[1]).count()[0].sort_values(ascending=False)
meta_serial.head(10)

对 meta serail 进行排序与汇总


1
7     7384
13    3670
27    2599
8     2011
28    1509
20    1397
6     1071
9      938
18     937
16     782
Name: 0, dtype: int64

## 选择 COUNT 值，使其筛选后， 接近 FilmTrust 数据规模

In [8]:
COUNT=500
filter_meta_serial =  meta_serial[meta_serial < COUNT]
print("保留 %s 个 meta, 原始 %s 个 meta" % (filter_meta_serial.size, meta_serial.size))

# 将 meta count 值修改为value 值, 便于使用 is_in 筛选
for key in filter_meta_serial.keys():
    filter_meta_serial[key]=key
dbm_meta = all_meta[all_meta[1].isin(filter_meta_serial)]
print_dataframe_stats(dbm_meta, dpath="dbm 保留 COUNT < %s 的 meta 数据 " % COUNT)
dbm_rating = dbm_rating_trust[dbm_rating_trust[1].isin(dbm_meta[0]) ]
print_dataframe_stats(dbm_rating, dpath="dbm 最终保留有 trust 和 filter meta 数据的rating")

utility.cuc_stat.py - INFO - === 数据集评分统计信息 dbm 保留 COUNT < 500 的 meta 数据  ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 2747 *       25        3235  4.710593  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 dbm 最终保留有 trust 和 filter meta 数据的rating ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 1384 *     2070       43873  1.531408  


保留 25 个 meta, 原始 38 个 meta


## 注意这里需要统一 user/item 的对照，否则可能会计算 key 错误

In [9]:
item_ids = pd.Series(list(set(dbm_meta[0]).intersection(set(all_rating[1].unique()))))
user_ids = pd.Series(list(set(dbm_rating_trust[0]).intersection(set(all_rating[0].unique()))))
dbm_rating = dbm_rating_trust[dbm_rating_trust[0].isin(user_ids)]
dbm_rating = dbm_rating[dbm_rating[1].isin(item_ids)]
# 统一 item 映射尺寸
dbm_meta = dbm_meta[dbm_meta[0].isin(dbm_rating[1])]
# 统一 trust 映射尺寸
dbm_trust = dbm_trust[dbm_trust[0].isin(dbm_rating[0])]
# print_dataframe_stats(dbm_rating, dpath="dbm 最终保留有 trust 和 filter meta 数据的rating")

## 写入新的 dbm 数据集

In [10]:
config2 = ConfigX(dataset_name="dbm")

print ("\n config2 预设值的文件信息为 ")
print (config2.trust_path)
print (config2.rating_path)
print (config2.meta_path)
print_dataframe_stats(dbm_rating, dpath="dbm rating")
print_dataframe_stats(dbm_trust, dpath="dbm trust")
print_dataframe_stats(dbm_meta, dpath="dbm meta")

utility.cuc_stat.py - INFO - === 数据集评分统计信息 dbm rating ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 1384 *     2070       43873  1.531408  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 dbm trust ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 1384 *     1764        2618  0.107235  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 dbm meta ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 2070 *       25        2440  4.714976  



 config2 预设值的文件信息为 
../data/dbm_trust.txt
../data/dbm_ratings.txt
../data/dbm_meta.txt


In [11]:
# 写入
dbm_rating.to_csv(config2.rating_path, sep=' ', header=None, index=None)
dbm_trust.to_csv(config2.trust_path, sep=' ', header=None, index=None)
dbm_meta.to_csv(config2.meta_path, sep=' ', header=None, index=None)
print("保存 %s 数据集 成功，请修改 configx.py self.dataset_name 以及 cross_validate.py 以进行实验 .. " % config2.dataset_name)

print_data_file_stats(config2.rating_path)
print_data_file_stats(config2.trust_path)
print_data_file_stats(config2.meta_path)

utility.cuc_stat.py - INFO - === 数据集评分统计信息 ../data/dbm_ratings.txt ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 1384 *     2070       43873  1.531408  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 ../data/dbm_trust.txt ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 1384 *     1764        2618  0.107235  
utility.cuc_stat.py - INFO - === 数据集评分统计信息 ../data/dbm_meta.txt ===
utility.cuc_stat.py - INFO -  m    *     n            数据量   稀疏度 (%) 
utility.cuc_stat.py - INFO - 2070 *       25        2440  4.714976  


保存 dbm 数据集 成功，请修改 configx.py self.dataset_name 以及 cross_validate.py 以进行实验 .. 
